In [1]:
import numpy as np
from numpy import linalg as LA

import sys
import re
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)
sc.setLogLevel('WARN')

loc = "./graph.txt"
#loc = sys.argv[1]


In [3]:
beta = 0.9
n = 1000

In [4]:
lines = sc.textFile(loc, 1)
line1 = lines.map(lambda l: re.split('\t',l))
line2 = line1.map(lambda x: (int(x[0]), int(x[1]))).distinct()
graph = line2.collect()

In [5]:
M = np.zeros((n, n), dtype = float)

for i in graph:
    M[i[1]-1, i[0]-1] = 1

Index of graph is 1 to 1000, but numpy array starts from 0.

In [6]:
num_linked = list()
for vec in M.transpose():
    c = 0
    for i in vec:
        if i==1:
            c+=1
    num_linked.append(c)

In [7]:
for i in range(n):
    M[:,i] /= num_linked[i]

# Method #1, Using Pyspark

In [8]:
Mat = sc.parallelize(M)

In [9]:
count = 0
_v = np.ones((n, 1), dtype = float) / n
while(count < 50):
    vec = Mat.flatMap(lambda m: np.matmul(m, _v))
    vec_list = vec.collect()
    _v = np.asarray(vec_list)
    _v.shape = (1000, 1)
    count += 1

In [10]:
sc.stop()

# Method #2, Not using Pyspark

In [11]:
v = np.ones((n, 1), dtype = float) / n
e = np.ones((n, 1), dtype = float)

In [12]:
count = 0
while(count < 50):
    nextv = beta * np.matmul(M, v) + (1 - beta) * e / n
    v = nextv
    count +=1

# Postprocessing

In [13]:
result1 = list()
for i in range(n):
    result1.append((i+1, v[i][0]))

In [14]:
sorted_result1 = sorted(result1, key = lambda x: x[1], reverse=True)

In [15]:
result2 = list()
for i in range(n):
    result2.append((i+1, _v[i][0]))

In [16]:
sorted_result2 = sorted(result2, key = lambda x: x[1], reverse=True)

In [17]:
#for i in range(10):
#    print(sorted_result1[i][0], '\t', sorted_result1[i][1])

In [18]:
for i in range(10):
    print(sorted_result2[i][0], '\t', sorted_result2[i][1])

537 	 0.002317768147392944
263 	 0.0022954975147433778
965 	 0.0021890105523848747
243 	 0.0020974242493302965
255 	 0.0020787314135073233
285 	 0.0020349676456453887
16 	 0.002029053713720983
736 	 0.0020093037888945747
747 	 0.0020080350790072105
126 	 0.001991457813337111
